In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

file_path = r"D:\Downloads\Germany_Simplified_Data_Short.csv"
germany_data = pd.read_csv(file_path)
germany_data['Datetime'] = pd.to_datetime(germany_data['Datetime'], format='%d/%m/%Y %H:%M')
germany_data.set_index('Datetime', inplace=True)


def feature_engineering(df):
    df['lag_price_1'] = df['market_price'].shift(1)  
    df['volatility'] = df['market_price'].rolling(window=7).std() 
    return df.dropna()


data = pd.DataFrame({
    'market_price': germany_data['Price (EUR/MWHE)'],
    'temperature': germany_data['Temperature (Celcius)']
})
data = feature_engineering(data)


train_data = data.iloc[:-int(len(data) * 0.66)]
test_data = data.iloc[-int(len(data) * 0.66):]

alpha = 0.3 
ema_predictions = test_data['market_price'].ewm(alpha=alpha, adjust=False).mean()


predictions_df = pd.DataFrame({
    'Datetime': test_data.index,
    'Actual_Price': test_data['market_price'].values,
    'EMA_Predicted': ema_predictions.values
})
output_path = r"D:\Downloads\EMA_Prediction_Data.csv"
predictions_df.to_csv(output_path, index=False)

print(f"Predictions saved to '{output_path}'")

def evaluate_model(actual, predicted, model_name):
    mse = ((actual - predicted) ** 2).mean()
    mae = np.abs(actual - predicted).mean()
    print(f"{model_name} - Mean Squared Error (MSE): {mse:.4f}")
    print(f"{model_name} - Mean Absolute Error (MAE): {mae:.4f}")

evaluate_model(test_data['market_price'], ema_predictions, "Exponential Moving Average (EMA)")

def plot_predictions(actual, predicted, title):
    plt.figure(figsize=(12, 6))
    plt.plot(actual, label="Actual Price", color='blue', alpha=0.8)
    plt.plot(predicted, label="EMA Predicted", linestyle="--", color='orange')
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.title(title)
    plt.grid()
    plt.show()

plot_predictions(
    test_data['market_price'].values,
    ema_predictions.values,
    "Exponential Moving Average (EMA) vs Actual Prices"
)

start_date = '2023-02-28 05:00:00'
end_date = '2023-03-03 05:00:00'
date_mask = (predictions_df['Datetime'] >= start_date) & (predictions_df['Datetime'] <= end_date)
filtered_df = predictions_df[date_mask]

filtered_df['Datetime'] = pd.to_datetime(filtered_df['Datetime'])

def plot_individual_prediction(df, actual_col, predicted_col, title):
    plt.figure(figsize=(12, 6))
    plt.plot(df['Datetime'], df[actual_col], label="Actual Price", color="blue")
    plt.plot(df['Datetime'], df[predicted_col], label="EMA Predicted", linestyle="--", color="orange")
    plt.legend()
    plt.xlabel("Datetime")
    plt.ylabel("Price")
    plt.title(title)
    plt.grid()
    plt.show()

plot_individual_prediction(filtered_df, 'Actual_Price', 'EMA_Predicted', "Exponential Moving Average (EMA) vs Actual Prices")